In [17]:
import pandas as pd

distance = 25
params = 0.99

dataPath = '../../data/'
filePath = f'{dataPath}/processed/adjacency_matrix.csv'
distance_matrix = pd.read_csv(filePath, index_col=0)


distance_threshold = distance
local_params = params
adj_params = 1 - local_params

dm_with_threshhold = distance_matrix[distance_matrix < distance_threshold]


# dm_with_threshhold의 모든 가중치를 역수로 취하기.
# 가중치가 0일경우(diagonal만) 1로 변경
dm_with_threshhold = dm_with_threshhold.replace(0, 1)
dm_with_threshhold = 1 / dm_with_threshhold


import numpy as np

# 각 행의 합을 계산
# 자신과의 가중치는 1이므로 1을 빼줌
row_sums = dm_with_threshhold.sum(axis=1) - 1


In [18]:
# 각 인접 도시와의 가중치를 총합으로 나눔
normalized_df = dm_with_threshhold.div(row_sums, axis=0) * adj_params

In [19]:
# 본인 반영비율은 전에 설정한 params로 변경
# threshold보다 가까운 이웃이 없다면 본인 반영비율 1로 변경

for i in range(len(normalized_df)):
    if normalized_df.iloc[i, i] != np.inf:
        normalized_df.iloc[i, i] = local_params
    else:
        normalized_df.iloc[i, i] = 1



In [20]:
normalized_df.to_csv(f'{dataPath}/processed/weighted_df/normalized_df_{distance}_{params}.csv')

In [21]:
df = pd.read_csv(f'{dataPath}/processed/no_NaN_dataset_final.csv')

In [22]:
def find_cityname_in_df(df, cityname, year):
    # cityname에서 첫 두글자는 시도에, 나머지는 시군구에 저장함
    
    sido = cityname[:2]
    sigungu = cityname[2:]
    
    # sido와 sigungu를 모두 포함하는 행 찾기 꼭 일치하지 않아도 되고 일부만 포함해도 됨
    
    sido_df = df[df['행정구역(시도)'].str.contains(sido)]
    # sigungu_df = sido_df[sido_df['행정구역(시군구)'].str.contains(sigungu)]
    sigungu_df = sido_df[sido_df['행정구역(시군구)'] == sigungu]
    
    
    result = sigungu_df[sigungu_df['연도'] == year]
    
    if len(result) == 0:
        sido = cityname[:2]
        sigungu = cityname[2:]
        
        sido_df = df[df['행정구역(시도)'].str.contains(sido)]
        
        sigungu_df = sido_df[sido_df['행정구역(시군구)'].str.contains(sigungu)]
        
        result = sigungu_df[sigungu_df['연도'] == year]
        
    if len(result) != 1:
        print("길이는 : " + str(len(result)))
        print(result.head())

    return result
        
        


In [23]:
def find_normalizedf_row(do, city):
    return normalized_df.loc[do + city]


from tqdm import tqdm

processed_df = df.copy()
numeric_columns = df.select_dtypes(include=[np.number]).columns
numeric_columns = numeric_columns.drop('학령인구(명)')
error_city_list = []


for i in tqdm(range(len(df))):
    try:
        parts = df.iloc[i]['행정구역(시도)'].split()
        do = parts[0][:2]
        city = df.iloc[i]['행정구역(시군구)']
        year = df.iloc[i]['연도']

        adj_matrix = find_normalizedf_row(do, city)
        row_list = []
        
            
        for j in range(len(adj_matrix)):
            try:

                adj_city_name = adj_matrix.index[j]
                adj_city_weight = adj_matrix.iloc[j]
                if pd.isna(adj_city_weight):
                    continue
                
                

                row = find_cityname_in_df(df, adj_city_name, year)
                
                #tmp_df.insert(row)
                row[numeric_columns] *= adj_city_weight

                
                if(row.empty):
                    error_city_list.append(adj_city_name)
                    continue
                
                row_list.append(row)
            except KeyError:
                print("error 2: " + adj_matrix.index[j])
                continue
        
                
        # Sum all the rows in row_list
        new_row = pd.concat(row_list)

        
        new_row = new_row[numeric_columns]
        new_row_sum = new_row.sum()

        

        # Use .loc to avoid SettingWithCopyWarning
        processed_df.loc[processed_df.index[i], numeric_columns] = new_row_sum

    except Exception as e:
        print(f"error 3:  - {str(e)}")
        print(do + " " + city)
        continue


processed_df['연도'] = processed_df['연도'].round().astype(int)

  0%|          | 0/2899 [00:00<?, ?it/s]/var/folders/39/yhddl2n53db5kx8yl47wsn_m0000gn/T/ipykernel_48544/2421551742.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2010.0000000000002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  processed_df.loc[processed_df.index[i], numeric_columns] = new_row_sum
/var/folders/39/yhddl2n53db5kx8yl47wsn_m0000gn/T/ipykernel_48544/2421551742.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '172726.33930667822' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  processed_df.loc[processed_df.index[i], numeric_columns] = new_row_sum
/var/folders/39/yhddl2n53db5kx8yl47wsn_m0000gn/T/ipykernel_48544/2421551742.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Valu

In [24]:
processed_df.to_csv(f'{dataPath}/processed/distance_df/{distance}_{params}.csv', index=False)

In [25]:
processed_df.head()

,행정구역(시도),행정구역(시군구),연도,총인구수(명),유치원 수,초등학교 수,출생건수,사망건수,혼인건수,이혼건수,학령인구(명),총인구수-학령인구(명),행정구역
0,서울특별시,종로구,2010,172726.339307,17.207303,14.099772,1131.225264,858.836789,1027.057757,400.418411,29164,143148.034810,서울특별시 종로구
1,서울특별시,중구,2010,135371.879218,13.246087,12.118332,1193.321585,679.681237,1064.583520,382.528583,19628,115229.132986,서울특별시 중구
2,서울특별시,용산구,2010,246401.718734,19.199748,15.095855,2325.774097,1090.664931,1793.561191,531.297854,37318,208715.862336,서울특별시 용산구
3,서울특별시,성동구,2010,309936.996480,31.065775,20.036777,2892.327396,1229.538600,2309.245108,673.877745,51562,258176.195522,서울특별시 성동구
4,서울특별시,광진구,2010,375938.296324,41.959246,22.030707,3371.468618,1293.039174,2674.059212,811.703659,67422,308398.915752,서울특별시 광진구
